# **MIP2-Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3."""

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Given:

1. There are two types of food denoted by f = {f1, f2}

2. There are three types of beverage denoted by b = {b1, b2, b3}

3. Time period denoted by t in {1, ..., 12}

4. Maximum capacity of each product for storage denoted by CS = {580 units of f1, 687 units of f2, 599L of b1, 788L of b2, 294L of b3}

5. Maximum capacity of each product for production denoted by CP = {1080 units of f1, 908 units of f2, 408L of b1, 1000L of b2, 403L of b3}


Parameters: 

1. Fixed cost to produce each product in each time period (F_i,t) where i is product and t in time period

2. Variable cost to produce each product in each time period (V_i,t)

3. Cost to store each product (S_i,t)

4. Maximum demand for each product at each time period (D_i,t)

5. Revenue for each product in each time period (R_i,t)


Decision Variables (all >= 0):

1. x_{i,t} : amount of product i to produce at time t

2. y_{i,t} : amount of product i to store at time t

3. z_{i,t} : amount of product i to sell at time t

Obj

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is a sample Python code using Pyomo modeling language to solve the problem. Note that the sample data are created to be in line with the given constraints' assumption.

```python
from pyomo.environ import *

# Sample data
products = ['f1', 'f2', 'b1', 'b2', 'b3']
time_periods = range(1, 13)
fixed_cost = {(i,t):2 for i in products for t in time_periods}
variable_cost = {(i,t):3 for i in products for t in time_periods}
storage_cost = {(i,t):1 for i in products for t in time_periods}
demand = {(i,t):400 for i in products for t in time_periods}
revenue = {(i,t):5 for i in products for t in time_periods}
CS = {'f1': 580, 'f2': 687, 'b1': 599, 'b2': 788, 'b3': 294}
CP = {'f1': 1080, 'f2': 908, 'b1': 408, 'b2': 1000, 'b3': 403}

# Creating the model
model = ConcreteModel()

# Defining decision variables
model.x = Var(products, time_periods, within=NonNegativeReals)
model.y = Var(products, time_periods, within=NonNegativeReals)
model.z = Var(products, time_periods, within=NonNegativeReals)

# Defining objective function
model.objective = Objective(expr=sum((revenue[i,t]*model.z[i,t] - (fixed_cost[i,t] + variable_cost[i,t]*model.x[i,t] + 
                         storage_cost[i,t]*model.y[i,t])) 
                         for i in products for t in time_periods), sense=maximize)

# Defining constraints
model.prod_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.prod_capacity.add(expr= model.x[i,t] <= CP[i])
        
model.storage_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.storage_capacity.add(expr= model.y[i,t] <= CS[i])

model.demand = ConstraintList()
for i in products:
    for t in time_periods:
        model.demand.add(expr= model.z[i,t] <= demand[i,t])
        
model.balance = ConstraintList()
for i in products:
    for t in time_periods:
        model.balance.add(expr= model.x[i,t] + (model.y[i,t-1] if t>1 else 0) == model.z[i,t] + model.y[i,t])

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Analyze the result
for i in products:
    for t in time_periods:
        print(i, t, value(model.x[i,t]), value(model.y[i,t]), value(model.z[i,t]))

```

This Python Pyomo script first creates a concrete model, then define the decision variables, the objective function and the constraints. It finally solves the linear programming problem using the GLPK solver and finally prints out the solution. This script assumes that you have Pyomo and the GLPK solver installed on your computer.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Sample data
products = ['f1', 'f2', 'b1', 'b2', 'b3']
time_periods = range(1, 13)
fixed_cost = {(i,t):2 for i in products for t in time_periods}
variable_cost = {(i,t):3 for i in products for t in time_periods}
storage_cost = {(i,t):1 for i in products for t in time_periods}
demand = {(i,t):400 for i in products for t in time_periods}
revenue = {(i,t):5 for i in products for t in time_periods}
CS = {'f1': 580, 'f2': 687, 'b1': 599, 'b2': 788, 'b3': 294}
CP = {'f1': 1080, 'f2': 908, 'b1': 408, 'b2': 1000, 'b3': 403}

# Creating the model
model = ConcreteModel()

# Defining decision variables
model.x = Var(products, time_periods, within=NonNegativeReals)
model.y = Var(products, time_periods, within=NonNegativeReals)
model.z = Var(products, time_periods, within=NonNegativeReals)

# Defining objective function
model.objective = Objective(expr=sum((revenue[i,t]*model.z[i,t] - (fixed_cost[i,t] + variable_cost[i,t]*model.x[i,t] +
                         storage_cost[i,t]*model.y[i,t]))
                         for i in products for t in time_periods), sense=maximize)

# Defining constraints
model.prod_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.prod_capacity.add(expr= model.x[i,t] <= CP[i])

model.storage_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.storage_capacity.add(expr= model.y[i,t] <= CS[i])

model.demand = ConstraintList()
for i in products:
    for t in time_periods:
        model.demand.add(expr= model.z[i,t] <= demand[i,t])

model.balance = ConstraintList()
for i in products:
    for t in time_periods:
        model.balance.add(expr= model.x[i,t] + (model.y[i,t-1] if t>1 else 0) == model.z[i,t] + model.y[i,t])

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Analyze the result
for i in products:
    for t in time_periods:
        print(i, t, value(model.x[i,t]), value(model.y[i,t]), value(model.z[i,t]))

f1 1 400.0 0.0 400.0
f1 2 400.0 0.0 400.0
f1 3 400.0 0.0 400.0
f1 4 400.0 0.0 400.0
f1 5 400.0 0.0 400.0
f1 6 400.0 0.0 400.0
f1 7 400.0 0.0 400.0
f1 8 400.0 0.0 400.0
f1 9 400.0 0.0 400.0
f1 10 400.0 0.0 400.0
f1 11 400.0 0.0 400.0
f1 12 400.0 0.0 400.0
f2 1 400.0 0.0 400.0
f2 2 400.0 0.0 400.0
f2 3 400.0 0.0 400.0
f2 4 400.0 0.0 400.0
f2 5 400.0 0.0 400.0
f2 6 400.0 0.0 400.0
f2 7 400.0 0.0 400.0
f2 8 400.0 0.0 400.0
f2 9 400.0 0.0 400.0
f2 10 400.0 0.0 400.0
f2 11 400.0 0.0 400.0
f2 12 400.0 0.0 400.0
b1 1 400.0 0.0 400.0
b1 2 400.0 0.0 400.0
b1 3 400.0 0.0 400.0
b1 4 400.0 0.0 400.0
b1 5 400.0 0.0 400.0
b1 6 400.0 0.0 400.0
b1 7 400.0 0.0 400.0
b1 8 400.0 0.0 400.0
b1 9 400.0 0.0 400.0
b1 10 400.0 0.0 400.0
b1 11 400.0 0.0 400.0
b1 12 400.0 0.0 400.0
b2 1 400.0 0.0 400.0
b2 2 400.0 0.0 400.0
b2 3 400.0 0.0 400.0
b2 4 400.0 0.0 400.0
b2 5 400.0 0.0 400.0
b2 6 400.0 0.0 400.0
b2 7 400.0 0.0 400.0
b2 8 400.0 0.0 400.0
b2 9 400.0 0.0 400.0
b2 10 400.0 0.0 400.0
b2 11 400.0 0.0 400.0
b2

In [ ]:
from pyomo.environ import *
import pandas as pd

# data inputted by human
products = range(1,6)
time_periods = range(1, 13)


fixedc = pd.read_csv("fixed_cost_production.csv")
fixedc.index +=1
fixedc = fixedc.drop("Unnamed: 0", axis = 1)
fixedc.columns = fixedc.columns.astype(int)
fixed_cost = fixedc.stack().to_dict()



varc = pd.read_csv("variable_cost_production.csv")
varc.index +=1
varc = varc.drop("Unnamed: 0", axis = 1)
varc.columns = varc.columns.astype(int)
variable_cost = varc.stack().to_dict()


stoc = pd.read_csv("variable_cost_storage.csv")
stoc.index +=1
stoc = stoc.drop("Unnamed: 0", axis = 1)
stoc.columns = stoc.columns.astype(int)
storage_cost = stoc.stack().to_dict()


demc = pd.read_csv("demand.csv")
demc.index +=1
demc  = demc.drop("Unnamed: 0", axis = 1)
demc.columns = demc.columns.astype(int)
demand = demc.stack().to_dict()

rev = pd.read_csv("revenue.csv")
rev.index +=1
rev = rev.drop("Unnamed: 0", axis = 1)
rev.columns = rev.columns.astype(int)
revenue = rev.stack().to_dict()


#end

CS = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
CP = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Creating the model
model = ConcreteModel()

# Defining decision variables
model.x = Var(products, time_periods, within=NonNegativeReals)
model.y = Var(products, time_periods, within=NonNegativeReals)
model.z = Var(products, time_periods, within=NonNegativeReals)

# Defining objective function
model.objective = Objective(expr=sum((revenue[i,t]*model.z[i,t] - (fixed_cost[i,t] + variable_cost[i,t]*model.x[i,t] +
                         storage_cost[i,t]*model.y[i,t]))
                         for i in products for t in time_periods), sense=maximize)

# Defining constraints
model.prod_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.prod_capacity.add(expr= model.x[i,t] <= CP[i])

model.storage_capacity = ConstraintList()
for i in products:
    for t in time_periods:
        model.storage_capacity.add(expr= model.y[i,t] <= CS[i])

model.demand = ConstraintList()
for i in products:
    for t in time_periods:
        model.demand.add(expr= model.z[i,t] <= demand[i,t])

model.balance = ConstraintList()
for i in products:
    for t in time_periods:
        model.balance.add(expr= model.x[i,t] + (model.y[i,t-1] if t>1 else 0) == model.z[i,t] + model.y[i,t])

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Analyze the result
print(model.objective())
for i in products:
    for t in time_periods:
        print(i, t, value(model.x[i,t]), value(model.y[i,t]), value(model.z[i,t]))


-8402.393900000001
1 1 4.61 0.0 4.61
1 2 4.93 0.0 4.93
1 3 4.92 0.0 4.92
1 4 4.71 0.0 4.71
1 5 4.87 0.0 4.87
1 6 4.6 0.0 4.6
1 7 5.02 0.0 5.02
1 8 4.86 0.0 4.86
1 9 4.87 0.0 4.87
1 10 4.9 0.0 4.9
1 11 4.67 0.0 4.67
1 12 5.2 0.0 5.2
2 1 3.79 0.0 3.79
2 2 4.2 0.0 4.2
2 3 2.81 0.0 2.81
2 4 3.89 0.0 3.89
2 5 2.52 0.0 2.52
2 6 5.21 0.0 5.21
2 7 4.5 0.0 4.5
2 8 4.1 0.0 4.1
2 9 1.84 0.0 1.84
2 10 4.66 0.0 4.66
2 11 4.53 0.0 4.53
2 12 4.16 0.0 4.16
3 1 5.68 0.0 5.68
3 2 8.2 0.0 8.2
3 3 7.85 0.0 7.85
3 4 5.83 0.0 5.83
3 5 8.05 0.0 8.05
3 6 6.13 0.0 6.13
3 7 6.38 0.0 6.38
3 8 7.3 0.0 7.3
3 9 6.75 0.0 6.75
3 10 5.83 0.0 5.83
3 11 5.86 0.0 5.86
3 12 5.85 0.0 5.85
4 1 2.71 0.0 2.71
4 2 2.91 0.0 2.91
4 3 2.35 0.0 2.35
4 4 2.98 0.0 2.98
4 5 2.53 0.0 2.53
4 6 2.4 0.0 2.4
4 7 2.91 0.0 2.91
4 8 2.37 0.0 2.37
4 9 1.19 0.0 1.19
4 10 5.14 0.0 5.14
4 11 2.77 0.0 2.77
4 12 2.69 0.0 2.69
5 1 3.93 0.0 3.93
5 2 4.0 0.0 4.0
5 3 4.04 0.0 4.04
5 4 3.99 0.0 3.99
5 5 4.13 0.0 4.13
5 6 3.85 0.0 3.85
5 7 3.94 0.0 3.94

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**